<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/Dart_Doc2vec/Dart%EA%B3%B5%EC%8B%9C%EC%9E%90%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd/content/drive/Shareddrives/텍스트 마이닝 프로젝트/경쟁사 찾기

In [ ]:
#!pip install dart-fss
!pip install opendartreader

In [ ]:
#dart api 인증키 : c585539e39b9f04f5aec7f95223bf660c253a73c
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [ ]:
#pip install --upgrade opendartreader

In [ ]:
import OpenDartReader

### 0. 객체 생성 ###
# 객체 생성 (API KEY 지정)
api_key = 'c585539e39b9f04f5aec7f95223bf660c253a73c'

dart = OpenDartReader(api_key)



# 전처리: 불용어 처리 및 토큰화

In [ ]:
# 불용어 사전
stop_words=pd.read_csv('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/stopwords.txt',sep='\t')

In [ ]:
!pip install konlpy

# 삼성전자 사업개요서 추출

In [ ]:
report = dart.list('삼성전자', start='2023', kind='A', final=False)
report_nm =report[report['report_nm'].str.contains('사업보고서')]
report_num=report_nm['rcept_no']
report_num
for j in report_num:
    sub_report=dart.sub_docs(j)
    report_url =sub_report[sub_report['title'].str.contains('사업의 개요')]
    report_url2=report_url['url']
for k in report_url2:
    url = k
    soup = BeautifulSoup(requests.get(url).text)

text=soup.text


In [ ]:
text

In [ ]:
report = dart.list('엔에이치스팩25호', start='2023', kind='A', final=False)
report_nm =report[report['report_nm'].str.contains('사업보고서')]
report_num=report_nm['rcept_no']
num=report_num.iloc[0]
sub_report=dart.sub_docs(num)
report_url =sub_report[sub_report['title'].str.contains('사업의 개요')]
report_url2=report_url['url'].iloc[0]
report_url2

In [ ]:
url=report_url2
soup = BeautifulSoup(requests.get(url).text)
text=soup.text

In [ ]:
cor_text=preprocessing(text)
del cor_text[0]
del cor_text[2]

import pickle
with open("삼성전자 사업개요.pkl","wb") as f:
    pickle.dump(samsung_text, f)

# 전체 상장기업 사업개요서 가져오기

In [ ]:
cor_list=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/상장법인목록.xlsx')

In [ ]:
document=[]
company_name=cor_list['회사명'][1000:]
for i in tqdm(company_name):
    try:
        report = dart.list(i, start='2023', kind='A', final=False)
        report_nm =report[report['report_nm'].str.contains('사업보고서')]
        report_num=report_nm['rcept_no']
        num=report_num.iloc[0]
        sub_report=dart.sub_docs(num)
        report_url =sub_report[sub_report['title'].str.contains('사업의 개요')]
        report_url2=report_url['url'].iloc[0]
        url=report_url2
        soup = BeautifulSoup(requests.get(url).text)
        document.append(soup.text)
    except:
        document.append('na')
document

In [ ]:
company_name = company_name.reset_index(drop=True)
df=pd.DataFrame({'company1':company_name,'document1':document})
df.to_csv('Doc2vec할 데이터.csv')

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [ ]:
okt = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['document1']
  tag = row['company1']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=okt.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

In [ ]:
tagged_corpus_list[0]

In [ ]:
from gensim.models import doc2vec

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
#print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/경쟁사 찾기/dart.doc2vec')

In [ ]:
similar_doc = model.docvecs.most_similar('삼성전자 반도체 사업을 하고 있다.')
print(similar_doc)

In [ ]:
model = doc2vec.Doc2Vec(vector_size=3, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
#print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')